In [5]:
import MeCab

m_t = MeCab.Tagger('-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

text = '機械学習が好きです。'

print(m_t.parse(text))

機械学習	キカイガクシュウ	機械学習	名詞-固有名詞-一般		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [6]:
# 単語分割する関数を定義
m_t = MeCab.Tagger('-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

def tokenizer_mecab(text):
    text = m_t.parse(text)
    ret = text.strip().split()
    return ret

text = '機械学習が好きです。'
print(tokenizer_mecab(text))

['機械学習', 'が', '好き', 'です', '。']


In [9]:
#単語分割にはJanomeを使用
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]

In [10]:
#前処理として正規化をする関数を定義
import re

def preprocessing_text(text):
    # 半角・全角の統一
    # 今回は無視
    
    # 英語の小文字化
    # 今回はここでは無視
    # output = output.lower()
    
    # 改行, 半角スペース, 全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)
    
    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)
    
    # 記号と数字の除去
    # 今回は無視. 半角記号, 数字, 英字
    # 今回は無視. 全角記号
    
    # 特定文字を正規表現で置換する
    # 今回は無視
    
    return text

In [11]:
# 前処理とJanomeの単語分割を合わせた関数を定義する

def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text) #前処理の正規化
    ret = tokenizer_janome(text) #Janomeの単語分割
    return ret

# 動作確認
text = '昨日は　とても暑く、気温が36度もあった。'
print(tokenizer_with_preprocessing(text))

['昨日', 'は', 'とても', '暑く', '、', '気温', 'が', '00', '度', 'も', 'あっ', 'た', '。']


In [13]:
import torchtext

# tsvやcsvデータを読み込んだ時に, 読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                           use_vocab=True, lower=True, include_lengths=True,
                           batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

In [14]:
# フォルダ「data」から各tsvファイルを読み込み, Datasetにします
# 1行がTEXTとLABELで区切られていることをfieldsで指示します
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('text', TEXT), ('label', LABEL)])

# 動作確認
print('訓練データの数', len(train_ds))
print('1つ目の訓練データ', vars(train_ds[0]))
print('2つ目の訓練データ', vars(train_ds[1]))

訓練データの数 4
1つ目の訓練データ {'text': ['王', 'と', '王子', 'と', '女王', 'と', '姫', 'と', '男性', 'と', '女性', 'が', 'い', 'まし', 'た', '。'], 'label': '0'}
2つ目の訓練データ {'text': ['機械', '学習', 'が', '好き', 'です', '。'], 'label': '1'}


In [15]:
# ボキャブラリーを作成
# 訓練データtrainの単語からmin_freq以上の頻度の単語を使用してボキャブラリー(単語集)を構築
TEXT.build_vocab(train_ds, min_freq=1)

# 訓練データ内の単語と頻度を出力(頻度min_freqより大きいものが出力される)
TEXT.vocab.freqs

Counter({'王': 1,
         'と': 5,
         '王子': 1,
         '女王': 1,
         '姫': 1,
         '男性': 1,
         '女性': 1,
         'が': 3,
         'い': 1,
         'まし': 1,
         'た': 1,
         '。': 4,
         '機械': 1,
         '学習': 1,
         '好き': 1,
         'です': 1,
         '本章': 2,
         'から': 1,
         '自然': 1,
         '言語': 1,
         '処理': 1,
         'に': 1,
         '取り組み': 1,
         'ます': 2,
         'で': 1,
         'は': 1,
         '商品': 1,
         'レビュー': 1,
         'の': 4,
         '短い': 1,
         '文章': 4,
         'に対して': 1,
         '、': 3,
         'その': 1,
         'ネガティブ': 1,
         'な': 4,
         '評価': 2,
         'を': 3,
         'し': 3,
         'て': 2,
         'いる': 2,
         'か': 2,
         'ポジティブ': 1,
         '0': 1,
         '値': 1,
         'クラス': 1,
         '分類': 2,
         'する': 1,
         'モデル': 1,
         '構築': 1})

In [17]:
# ボキャブラリーの単語をidに変換した結果を出力
# 頻度がmin_freqより小さい場合は未知語<unk>になる

TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x121cdd400>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '取り組み': 37,
             '商品': 38,
        

In [19]:
# DataLoaderを作成します (torchtextの文脈では単純にiteratorと呼ばれています)
train_dl = torchtext.data.Iterator(train_ds, batch_size=2, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=2, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=2, train=False, sort=False)

# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.text)
print(batch.label)

(tensor([[46,  2, 47,  2, 40,  2, 42,  2, 48,  2, 39,  8, 19, 29, 23,  3,  1,  1,
          1,  1,  1,  1,  1,  1,  1],
        [45, 43,  8, 41, 25,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1]]), tensor([16,  6]))
tensor([0, 1])


In [22]:
from gensim.models import KeyedVectors

# 一度gensimライブラリで読み込んで, word2vecのformatで保存する
model = KeyedVectors.load_word2vec_format(
    './data/entity_vector/entity_vector.model.bin', binary=True)

model.wv.save_word2vec_format('./data/japanese_word2vec_vectors.vec')

/Users/yohei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [23]:
# torchtextで単語ベクトルとして読み込む
from torchtext.vocab import Vectors

japanese_word2vec_vectors = Vectors(
    name='./data/japanese_word2vec_vectors.vec')

# 単語ベクトルの中身を確認
print("1単語を表現する次元数：", japanese_word2vec_vectors.dim)
print("単語数：", len(japanese_word2vec_vectors.itos))

100%|█████████▉| 1014656/1015474 [01:33<00:00, 12875.82it/s]

1単語を表現する次元数： 200
単語数： 1015474


In [24]:
# ベクトル化したバージョンのボキャブラリーを作成
TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認
print(TEXT.vocab.vectors.shape)
TEXT.vocab.vectors

torch.Size([52, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.6023, -2.6357, -2.5822,  ...,  0.6953, -1.4977,  1.4752],
        ...,
        [-1.5885,  0.1614, -0.6029,  ..., -1.7545, -1.2462,  2.3034],
        [-0.3807,  0.4007, -1.6304,  ..., -2.2620, -0.5581,  0.9880],
        [-3.2000,  2.9676, -0.9007,  ...,  1.8192, -0.4641,  0.3200]])

In [25]:
# ボキャブラリーの単語の順番を確認
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x1fb4529e8>>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             '、': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '取り組み': 37,
             '商品': 38,
        

In [27]:
# 姫 - 女性 + 男性のベクトルがどれと似ているか確認
import torch.nn.functional as F

# 姫 - 女性 - 男性
tensor_calc = TEXT.vocab.vectors[42] - TEXT.vocab.vectors[39] + TEXT.vocab.vectors[48]

# コサイン類似度を計算
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[40], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[46], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[47], dim=0))
print("機械", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))

女王 tensor(0.3840)
王 tensor(0.3669)
王子 tensor(0.5489)
機械 tensor(-0.0867)


fastTextについては(日本語の精度は低いらしいので)省略.

In [29]:
import glob
import os
import io
import string

# 訓練データのtsvファイルを作成
f = open('./data/IMDb_train.tsv', 'w')

path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding='utf-8') as ff:
        text = ff.readline()
        
        # タブがあれば消す
        text = text.replace('\t', '')
        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)
        
path = './data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding='utf-8') as ff:
        text = ff.readline()
        
        # タブがあれば消す
        text = text.replace('\t', '')
        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)
f.close()

In [30]:
# テストデータのtsvファイルを作成
f = open('./data/IMDb_test.tsv', 'w')

path = './data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding='utf-8') as ff:
        text = ff.readline()
        
        # タブがあれば消す
        text = text.replace('\t', '')
        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)
        
path = './data/aclImdb/test/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding='utf-8') as ff:
        text = ff.readline()
        
        # タブがあれば消す
        text = text.replace('\t', '')
        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)
f.close()

In [31]:
# punctuationは句点のこと
print('区切り文字:', string.punctuation)

# 前処理
def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br \>', '', text)
    #カンマとピリオド以外の記号をスペースに変換
    for p in string.punctuation:
        if (p != '.') or (p != ','):
            continue
        else:
            text = text.replace(p, " ")
    #ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text
# 分かち書き
def tokenizer_punctuation(text):
    return text.strip().split()
# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret

# 動作確認
print(tokenizer_with_preprocessing('I like cats.'))

区切り文字: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cats', '.']


In [32]:
# 文章とラベルの両方に用意
max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                           use_vocab=True,
                           lower=True, include_lengths=True, batch_first=True,
                           fix_length=max_length, init_token="<cls>",
                           eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# init_token, eos_tokenは文頭・文末の単語を指す

In [33]:
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 動作確認
print('訓練および検証のデータ数:', len(train_val_ds))
print('1つ目の訓練および検証のデータ:', vars(train_val_ds[0]))

訓練および検証のデータ数: 25000
1つ目の訓練および検証のデータ: {'Text': ['for', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', '.', 'imagine', 'a', 'movie', 'where', 'joe', 'piscopo', 'is', 'actually', 'funny!', 'maureen', 'stapleton', 'is', 'a', 'scene', 'stealer', '.', 'the', 'moroni', 'character', 'is', 'an', 'absolute', 'scream', '.', 'watch', 'for', 'alan', '"the', 'skipper"', 'hale', 'jr', '.', 'as', 'a', 'police', 'sgt', '.'], 'Label': '1'}


In [34]:
import random
# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

# 動作確認
print('訓練データの数:', len(train_ds))
print('検証データの数:', len(val_ds))
print('訓練データの数:', vars(train_ds[0]))

訓練データの数: 20000
検証データの数: 5000
訓練データの数: {'Text': ['while', 'others', 'may', 'contend', 'that', 'by', 'viewing', 'other', 'works', 'by', 'bilal', ',', 'one', 'will', 'better', 'appreciate', 'this', 'movie', ',', 'it', 'does', 'fail', 'in', 'one', 'major', 'way', '.', 'it', 'does', 'not', 'stand', 'on', 'its', 'own', '.', 'the', 'plot', 'is', 'a', 'mishmash', 'that', 'is', 'confuses', 'symbolism', 'with', 'substance', '.', "here's", 'an', 'idea', 'start', 'with', 'a', 'definite', 'story', '.', 'then', 'craft', 'symbolism', 'around', 'it', '.', 'we', 'start', 'with', 'two', 'different', 'narratives', ',', 'this', 'female', 'that', 'is', 'somehow', 'turning', 'human', ',', 'a', '"god"', 'that', 'is', 'for', 'some', 'reason', 'being', 'judged', ',', 'but', 'getting', 'one', 'last', 'fling', 'on', 'earth', ',', 'and', 'this', 'mysterious', 'john', 'character', 'who', 'seems', 'to', 'be', 'developing', 'some', 'sort', 'of', '"resort"', 'just', 'beyond', 'the', 'bounds', 'of', 'the', 'city', '.'

In [35]:
# torchtextで単語ベクトルとして英語学習済みモデルを読み込む
from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')

# 単語ベクトルの中身を確認
print('1単語を表現する次元数:', english_fasttext_vectors.dim)
print('単語数:', len(english_fasttext_vectors.itos))


  0%|          | 0/999994 [00:00<?, ?it/s]Skipping token b'999994' with 1-dimensional vector [b'300']; likely a header

 12%|█▏        | 117722/999994 [00:14<01:49, 8088.15it/s]


 24%|██▍       | 243241/999994 [00:28<01:27, 8656.60it/s]


 38%|███▊      | 375748/999994 [00:42<01:05, 9581.41it/s] 


 51%|█████     | 510192/999994 [00:55<00:49, 9926.42it/s] 


 65%|██████▍   | 645023/999994 [01:09<00:35, 9926.29it/s] 


 78%|███████▊  | 776912/999994 [01:23<00:21, 10148.74it/s]


 91%|█████████▏| 913376/999994 [01:36<00:08, 9898.06it/s]


100%|█████████▉| 999423/999994 [01:45<00:00, 10148.89it/s]

1単語を表現する次元数: 300
単語数: 999994



100%|█████████▉| 999423/999994 [02:03<00:00, 10148.89it/s]

In [36]:
# ベクトル化したバージョンのボキャブラリーを作成
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# ボキャブラリーのベクトルを確認
print(TEXT.vocab.vectors.shape)
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認
TEXT.vocab.stoi

torch.Size([19531, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x271fbf518>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             '.': 4,
             'the': 5,
             ',': 6,
             'and': 7,
             'a': 8,
             'of': 9,
             'to': 10,
             'is': 11,
             'in': 12,
             'it': 13,
             'this': 14,
             'i': 15,
             'that': 16,
             '/><br': 17,
             'was': 18,
             '<br': 19,
             'as': 20,
             'for': 21,
             'with': 22,
             'movie': 23,
             'but': 24,
             'film': 25,
             'on': 26,
             'not': 27,
             'are': 28,
             'his': 29,
             'you': 30,
             'have': 31,
             'be': 32,
             'he': 33,
             'one': 34,
             'at': 35,
             'all': 36,
             'by': 37

In [37]:
# DataLoaderを作成
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)
val_dl = torchtext.data.Iterator(val_ds, batch_size=24, train=False, sort=False)
test_dl = torchtext.data.Iterator(test_ds, batch_size=24, train=False, sort=False)

# 動作確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[    2,    14,    11,  ...,     1,     1,     1],
        [    2,   137,   116,  ...,     1,     1,     1],
        [    2, 14056,  1027,  ...,   342,    10,     3],
        ...,
        [    2,    49,    80,  ...,     1,     1,     1],
        [    2,    15,    67,  ...,    57,     0,     3],
        [    2,    15,  1791,  ...,     1,     1,     1]]), tensor([185,  59, 256, 106, 139,  50, 200, 256, 147, 256, 256, 207, 256, 172,
        151, 242, 235, 256, 136, 151, 256, 154, 256, 194]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])


In [59]:
import numpy as np
import torch
import torch.nn as nn
import math

class Embedder(nn.Module):
    # idで示されている単語をベクトルに変換
    def __init__(self, text_embedding_vectors):
        super(Embedder, self).__init__()
        self.embeddings = nn.Embedding.from_pretrained(
            embeddings=text_embedding_vectors, freeze=True)
        # freeze=Trueによりバックプロパゲーションで更新されず変化しなくなる
    def forward(self, x):
        x_vec = self.embeddings(x)
        return x_vec

In [60]:
# 動作確認
# 前節のDataLoaderなどを取得
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT
train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(
    max_length=256, batch_size=24)

# ミニバッチの用意
batch = next(iter(train_dl))

# モデル構築
net1 = Embedder(TEXT.vocab.vectors)

# 入出力
x = batch.Text[0]
x1 = net1(x) # 単語をベクトルに

print('入力のテンソルサイズ:', x.shape)
print('出力のテンソルサイズ:', x1.shape)

入力のテンソルサイズ: torch.Size([24, 256])
出力のテンソルサイズ: torch.Size([24, 256, 300])


In [61]:
class PositionalEncoder(nn.Module):
    # 入出力された単語の位置を示すベクトル情報を付加する
    def __init__(self, d_model=300, max_seq_len=256):
        super().__init__()
        self.d_model = d_model # 単語ベクトルの次元数
        # 単語の順番(pos)と埋め込みベクトルの次元の位置(i)によって一意に定まる値の表をpeとする
        pe = torch.zeros(max_seq_len, d_model)
        # GPUが利用できる場合はGPUに送る
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        pe = pe.to(device)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i+1] = math.cos(pos / (10000 ** ((2 * (i+1))/d_model)))
        # 表peの先頭に, ミニバッチ次元を足す
        self.pe = pe.unsqueeze(0)
        # 勾配は計算しない
        self.pe.requires_grad = False
    
    def forward(self, x):
        # 入力xとPositional Encodingを足し算する
        # xがpeよりも小さいので大きくする
        ret = math.sqrt(self.d_model)*x + self.pe
        return ret

In [62]:
# 動作確認

# モデル構築
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)

# 入出力
x = batch.Text[0]
x1 = net1(x) # 単語をベクトルに
x2 = net2(x1)

print('入力のテンソルサイズ:', x1.shape)
print('出力のテンソルサイズ:', x2.shape)

入力のテンソルサイズ: torch.Size([24, 256, 300])
出力のテンソルサイズ: torch.Size([24, 256, 300])


In [63]:
class Attention(nn.Module):
    # Transformerは本当はマルチヘッドAttention, 今回はシングルヘッドで実装
    def __init__(self, d_model=300):
        super().__init__()
        # SAGANでは1dConvを使用したが, 今回は全結合層で特徴量を変換
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        # 出力時に使用する全結合層
        self.out = nn.Linear(d_model, d_model)
        # Attentionの大きさ調節の変数
        self.d_k = d_model
    def forward(self, q, k, v, mask):
        # 全結合層で特徴量を変換
        k = self.k_linear(k)
        q = self.q_linear(q)
        v = self.v_linear(v)
        # Attentionの値を計算する
        # 各値を足し算すると大きすぎるので, root(d_k)で割って調整
        weights = torch.matmul(q, k.transpose(1,2) / math.sqrt(self.d_k))
        # ここでmaskを計算
        mask = mask.unsqueeze(1)
        weights = weights.masked_fill(mask==0, -1e9)
        # softmaxで規格化する
        normalized_weights = F.softmax(weights, dim=-1)
        # Attentionをvalueと掛け算
        output = torch.matmul(normalized_weights, v)
        # 全結合層で特徴量を変換
        output = self.out(output)
        return output, normalized_weights

In [64]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=1024, dropout=0.1):
        # Attention層から出力を単純に全結合層2つで特徴量を変換するだけのユニット
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.linear_1(x)
        x = self.dropout(F.relu(x))
        x = self.linear_2(x)
        return x

class TransformerBlock(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        # LayerNormalization層
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)
        # Attention層
        self.attn = Attention(d_model)
        # Attentionのあとの全結合層2つ
        self.ff = FeedForward(d_model)
        # Dropout
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
    def forward(self, x, mask):
        # 正規化とAttention
        x_normalized = self.norm_1(x)
        output, normalized_weights = self.attn(
            x_normalized, x_normalized, x_normalized, mask)
        x2 = x + self.dropout_1(output)
        # 正規化と全結合層
        x_normalized2 = self.norm_2(x2)
        output = x2 + self.dropout_2(self.ff(x_normalized2))
        return output, normalized_weights

In [65]:
# 動作環境
# モデル構築
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)
net3 = TransformerBlock(d_model=300)

# maskの作成
x = batch.Text[0]
input_pad = 1 #単語のIDにおいて, '<pad>':1なので
input_mask = (x != input_pad)
print(input_mask[0])

# 入出力
x1 = net1(x) # 単語をベクトルに
x2 = net2(x1) # Position情報を足し算
x3, normalized_weights = net3(x2, input_mask)

print('入力のテンソルのサイズ:', x2.shape)
print('出力のテンソルのサイズ:', x3.shape)
print('Attentionのサイズ:', normalized_weights.shape)

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True, False, False, 

In [66]:
class ClassificationHead(nn.Module):
    # Transformer_Blockの出力を使用し, 最後にクラス分類させる
    def __init__(self, d_model=300, output_dim=2):
        super().__init__()
        # 全結合層
        self.linear = nn.Linear(d_model, output_dim)
        # 重み初期化処理
        nn.init.normal_(self.linear.weight, std=0.02)
        nn.init.normal_(self.linear.bias, 0)
    def forward(self, x):
        x0 = x[:, 0, :] #各ミニバッチの先頭の単語の特徴量(300次元)を取り出す
        out = self.linear(x0)
        return out

In [67]:
# 最終的なTransformerモデルのクラス
class TransformerClassification(nn.Module):
    # Transformerでクラス分類させる
    def __init__(self, text_embedding_vectors, d_model=300, max_seq_len=256, output_dim=2):
        super().__init__()
        # モデル構築
        self.net1 = Embedder(text_embedding_vectors)
        self.net2 = PositionalEncoder(d_model=d_model, max_seq_len=max_seq_len)
        self.net3_1 = TransformerBlock(d_model=d_model)
        self.net3_2 = TransformerBlock(d_model=d_model)
        self.net4 = ClassificationHead(output_dim=output_dim, d_model=d_model)
        
    def forward(self, x, mask):
        x1 = self.net1(x) # 単語をベクトルに
        x2 = self.net2(x1) # Position情報を足し算
        x3_1, normalized_weights_1 = self.net3_1(x2, mask) # self-attentionで特徴量を変換
        x3_2, normalized_weights_2 = self.net3_2(x3_1, mask) # self-attentionで特徴量を変換
        x4 = self.net4(x3_2) #最終出力の0単語目を使用して分類0-1のスカラーを出力
        return x4, normalized_weights_1, normalized_weights_2

In [68]:
# 動作確認
# ミニバッチの用意
batch = next(iter(train_dl))
# モデル構築
net = TransformerClassification(text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)
# 入出力
x = batch.Text[0]
input_mask = (x != input_pad)
out, normalized_weights_1, normalized_weights_2 = net(x, input_mask)
print('出力のテンソルサイズ:', out.shape)
print('出力テンソルのsigmoid:', F.softmax(out, dim=1))

出力のテンソルサイズ: torch.Size([24, 2])
出力テンソルのsigmoid: tensor([[0.3906, 0.6094],
        [0.3909, 0.6091],
        [0.3638, 0.6362],
        [0.3591, 0.6409],
        [0.3078, 0.6922],
        [0.3501, 0.6499],
        [0.4085, 0.5915],
        [0.3433, 0.6567],
        [0.3473, 0.6527],
        [0.3505, 0.6495],
        [0.3474, 0.6526],
        [0.3534, 0.6466],
        [0.3887, 0.6113],
        [0.3148, 0.6852],
        [0.3833, 0.6167],
        [0.3612, 0.6388],
        [0.3490, 0.6510],
        [0.3628, 0.6372],
        [0.3655, 0.6345],
        [0.3490, 0.6510],
        [0.3153, 0.6847],
        [0.3617, 0.6383],
        [0.3549, 0.6451],
        [0.3697, 0.6303]], grad_fn=<SoftmaxBackward>)


In [73]:
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT

# 読み込み
train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(
    max_length=256, batch_size=64)
# 辞書オブジェクトにまとめる
dataloaders_dict = {'train': train_dl, 'val': val_dl}

from utils.transformer import TransformerClassification

# モデル構築
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256,
    output_dim=2)

# ネットワークの初期化を定義
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        # Linear層の初期化
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

# 訓練モードに設定
net.train()
# TransformerBlockモジュールを初期化実行
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)
print('ネットワークの初期化完了')

ネットワークの初期化完了


In [74]:
import torch.optim as optim
# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# 最適化手法の設定
learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [75]:
# モデルを学習させる関数を作成
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('使用デバイス:', device)
    print('-----start-----')
    # ネットワークをGPUへ
    net.to(device)
    # ネットワークがある程度固定であれば, 高速化させる
    torch.backends.cudnn.benchmark = True
    # epochのループ
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() # モデルを訓練モードに
            else:
                net.eval() # モデルを検証モードに
            epoch_loss = 0.0 # epochの損失和
            epoch_corrects = 0 # epochの正解数
            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLabelの辞書オブジェクト
                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device) # 文章
                labels = batch.Label.to(device) # ラベル
                # optimizerを初期化
                optimizer.zero_grad()
                # 順伝搬(forward)計算
                with torch.set_grad_enabled(phase == 'train'):
                    # mask作成
                    input_pad = 1 #単語のIDにおいて'<pad>':1のため
                    input_mask = (inputs != input_pad)
                    # Transformerに入力
                    outputs, _, _ = net(inputs, input_mask)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1) # ラベルを予測
                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0) # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)
                # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            print('Epoch {}/{} | {:^5} | Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs, phase, epoch_loss, epoch_acc))
    return net

In [76]:
# 学習・検証を実行する
num_epochs = 10
net_trained = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス: cpu
-----start-----


KeyboardInterrupt: 